<a href="https://colab.research.google.com/github/Smruthi3/END2/blob/main/Session7-Assignment/%20Assignment-Part2/Session7_Assignment_part2_QA_seq2seq_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.legacy.datasets import Multi30k
from torchtext.legacy.data import Field, BucketIterator

from torchtext.legacy import data

import spacy
import numpy as np

import random
import math
import time

In [33]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
#!pip install spacy --upgrade

In [34]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Data Preview : 
The Question/Answer dataset generated by students 
who took undergraduate natural language processing courses taught by Noah Smith 
at Carnegie Mellon and Rebecca Hwa at the University of Pittsburgh during 
Spring 2008, Spring 2009, and Spring 2010.

There are three directories, one for each year of students: S08, S09, and S10 and each of the folder have question_answer_pairs.txt file

This data set can be downloaded from http://www.cs.cmu.edu/~ark/QA-data/

These file contains the totaly 6 columns out of which question and answer are our point of interset

Also note that, there are frequently multiple lines with the same question, which appear if those questions were answered 
by multiple individuals. So duplicate questions should be retained

In [35]:
import os 
import pandas as pd
DATA_DIR = 'drive/My Drive/END2/Session7-Assignment/Assignment-part2/'

In [36]:
# import shutil
# shutil.unpack_archive(DATA_DIR+'Question_Answer_Dataset_v1.2.tar.gz', DATA_DIR+'QA_dataset')

In [37]:
import glob

import pandas as pd


QA_dataset_complete = pd.DataFrame()

# get all .txt files from all subdirectories

all_files = glob.glob(DATA_DIR+'QA_dataset/Question_Answer_Dataset_v1.2/*/*.txt')


for file in all_files:

  QA_dataset = pd.read_csv(file, sep='\t', encoding= 'ISO-8859-1')
                           #'unicode_escape',header=None,error_bad_lines=False,skiprows=[0])
  print(f'Complete file path :{file} | Number of records or dimentions : {QA_dataset.shape}')
  QA_dataset_complete=pd.concat([QA_dataset_complete,QA_dataset])


print(f'After combining all files total number of records or dimentions : {QA_dataset_complete.shape}') 

Complete file path :drive/My Drive/END2/Session7-Assignment/Assignment-part2/QA_dataset/Question_Answer_Dataset_v1.2/S09/question_answer_pairs.txt | Number of records or dimentions : (825, 6)
Complete file path :drive/My Drive/END2/Session7-Assignment/Assignment-part2/QA_dataset/Question_Answer_Dataset_v1.2/S08/question_answer_pairs.txt | Number of records or dimentions : (1715, 6)
Complete file path :drive/My Drive/END2/Session7-Assignment/Assignment-part2/QA_dataset/Question_Answer_Dataset_v1.2/S10/question_answer_pairs.txt | Number of records or dimentions : (1458, 6)
After combining all files total number of records or dimentions : (3998, 6)


In [38]:
#QA_dataset_complete = QA_dataset_complete.reset_index(drop=True)
QA_dataset_complete.head()

,ArticleTitle,Question,Answer,DifficultyFromQuestioner,DifficultyFromAnswerer,ArticleFile
0,Alessandro_Volta,Was Volta an Italian physicist?,yes,easy,easy,data/set4/a10
1,Alessandro_Volta,Was Volta an Italian physicist?,yes,easy,easy,data/set4/a10
2,Alessandro_Volta,Is Volta buried in the city of Pittsburgh?,no,easy,easy,data/set4/a10
3,Alessandro_Volta,Is Volta buried in the city of Pittsburgh?,no,easy,easy,data/set4/a10
4,Alessandro_Volta,Did Volta have a passion for the study of elec...,yes,easy,medium,data/set4/a10


In [39]:
QA_dataset_complete.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3998 entries, 0 to 3997
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   ArticleTitle              3998 non-null   object
 1   Question                  3961 non-null   object
 2   Answer                    3422 non-null   object
 3   DifficultyFromQuestioner  3043 non-null   object
 4   DifficultyFromAnswerer    3418 non-null   object
 5   ArticleFile               3996 non-null   object
dtypes: object(6)
memory usage: 187.5+ KB


In [40]:
QA_dataset_complete.isnull().sum()

ArticleTitle                  0
Question                     37
Answer                      576
DifficultyFromQuestioner    955
DifficultyFromAnswerer      580
ArticleFile                   2
dtype: int64

In [41]:
QA_dataset_complete = QA_dataset_complete.dropna()
QA_dataset_complete = QA_dataset_complete.reset_index(drop=True)
QA_dataset_complete.shape

(2733, 6)

In [ ]:
# %%bash
# python -m spacy download en
# python -m spacy download de

In [ ]:
# spacy_de = spacy.load('de_core_news_sm')
# spacy_en = spacy.load('en_core_web_sm')

In [ ]:
# def tokenize_de(text):
#     """
#     Tokenizes German text from a string into a list of strings (tokens) and reverses it
#     """
#     return [tok.text for tok in spacy_de.tokenizer(text)][::-1]

# def tokenize_en(text):
#     """
#     Tokenizes English text from a string into a list of strings (tokens)
#     """
#     return [tok.text for tok in spacy_en.tokenizer(text)]

### Defining Fields

In [42]:
question = Field(tokenize = 'spacy', 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

answer = Field(tokenize = 'spacy', 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)


fields = [('questions', question),('answers',answer)]

In [43]:
QA_dataset_examples = [data.Example.fromlist([str(QA_dataset_complete.Question[i]),str(QA_dataset_complete.Answer[i])],fields) for i in range(QA_dataset_complete.shape[0])]

In [44]:
QA_Dataset= data.Dataset(QA_dataset_examples, fields)

In [45]:
(train,test) = QA_Dataset.split(split_ratio=[0.70, 0.30], random_state=random.seed(SEED))

In [46]:
print(f"Toatal examples: {len(QA_Dataset.examples)}")
print(f"Number of training examples: {len(train.examples)}")
print(f"Number of testing examples: {len(test.examples)}")

Toatal examples: 2733
Number of training examples: 1913
Number of testing examples: 820


In [47]:
print(vars(train.examples[1]))

{'questions': ['what', 'do', 'tigers', "'", 'powerfully', 'built', 'legs', 'and', 'shoulders', 'allow', 'tigers', 'to', 'do', '?'], 'answers': ['they', 'have', 'the', 'ability', 'to', 'pull', 'down', 'prey', 'substantially', 'heavier', 'than', 'themselves', '.']}


### Building Vocabulary

In [48]:
question.build_vocab(train, min_freq = 2)
answer.build_vocab(train, min_freq = 2)

In [49]:
print(f"Unique tokens in question vocabulary: {len(question.vocab)}")
print(f"Unique tokens in answer vocabulary: {len(answer.vocab)}")

Unique tokens in question vocabulary: 1574
Unique tokens in answer vocabulary: 1140


In [50]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [51]:
BATCH_SIZE = 128

train_iterator, test_iterator = data.BucketIterator.splits(
    (train, test), 
    batch_size = BATCH_SIZE, 
    sort_key= lambda x: len(x.questions),
    sort_within_batch=True,
    device = device)

### Defining The model arrchitecture

In [52]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, (hidden, cell) = self.rnn(embedded)
        
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden, cell

In [53]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.fc_out = nn.Linear(hid_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, cell):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #n directions in the decoder will both always be 1, therefore:
        #hidden = [n layers, batch size, hid dim]
        #context = [n layers, batch size, hid dim]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
                
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #seq len and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [n layers, batch size, hid dim]
        #cell = [n layers, batch size, hid dim]
        
        prediction = self.fc_out(output.squeeze(0))
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden, cell

In [54]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(src)
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden and previous cell states
            #receive output tensor (predictions) and new hidden and cell states
            output, hidden, cell = self.decoder(input, hidden, cell)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1
        
        return outputs

In [55]:
INPUT_DIM = len(question.vocab)
OUTPUT_DIM = len(answer.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq(enc, dec, device).to(device)

In [56]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(1574, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(1140, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (fc_out): Linear(in_features=512, out_features=1140, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [57]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 8,636,020 trainable parameters


# Model Training and Evaluation

In [58]:
optimizer = optim.Adam(model.parameters())

In [59]:
TRG_PAD_IDX = answer.vocab.stoi[answer.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

### Training loop

In [60]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        ques = batch.questions
        ans = batch.answers
        
        optimizer.zero_grad()
        
        output = model(ques, ans)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        ans = ans[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, ans)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

### Evaluation loop

In [61]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            ques = batch.questions
            ans = batch.answers

            output = model(ques, ans, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            ans = ans[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, ans)
            
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [62]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

### Model training and Evaluation

In [63]:
N_EPOCHS = 10
CLIP = 1

best_test_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    test_loss = evaluate(model, test_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if test_loss < best_test_loss:
        best_test_loss = test_loss
        torch.save(model.state_dict(), DATA_DIR+'QA-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {test_loss:.3f} |  Val. PPL: {math.exp(test_loss):7.3f}')

Epoch: 01 | Time: 1m 14s
	Train Loss: 5.419 | Train PPL: 225.546
	 Val. Loss: 3.907 |  Val. PPL:  49.737
Epoch: 02 | Time: 1m 13s
	Train Loss: 4.526 | Train PPL:  92.351
	 Val. Loss: 3.791 |  Val. PPL:  44.310
Epoch: 03 | Time: 1m 13s
	Train Loss: 4.417 | Train PPL:  82.823
	 Val. Loss: 3.770 |  Val. PPL:  43.367
Epoch: 04 | Time: 1m 13s
	Train Loss: 4.355 | Train PPL:  77.882
	 Val. Loss: 3.783 |  Val. PPL:  43.962
Epoch: 05 | Time: 1m 11s
	Train Loss: 4.315 | Train PPL:  74.788
	 Val. Loss: 3.770 |  Val. PPL:  43.360
Epoch: 06 | Time: 1m 11s
	Train Loss: 4.283 | Train PPL:  72.487
	 Val. Loss: 3.803 |  Val. PPL:  44.827
Epoch: 07 | Time: 1m 13s
	Train Loss: 4.207 | Train PPL:  67.135
	 Val. Loss: 3.829 |  Val. PPL:  46.018
Epoch: 08 | Time: 1m 10s
	Train Loss: 4.186 | Train PPL:  65.775
	 Val. Loss: 3.819 |  Val. PPL:  45.552
Epoch: 09 | Time: 1m 11s
	Train Loss: 4.146 | Train PPL:  63.192
	 Val. Loss: 3.811 |  Val. PPL:  45.196
Epoch: 10 | Time: 1m 11s
	Train Loss: 4.119 | Train PPL